In [1]:
import time
from unityagents import UnityEnvironment
import numpy as np
import torch
import random
from agent import Agent
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
env = UnityEnvironment(file_name='Env_20_Agents/Reacher_Linux/Reacher.x86_64', no_graphics = True)

UnityTimeOutException: The Unity environment took too long to respond. Make sure that :
	 The environment does not need user interaction to launch
	 The Academy and the External Brain(s) are attached to objects in the Scene
	 The environment and the Python interface have compatible versions.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

In [ ]:
#definition hyperparameters and trainingsconditions
n_episodes = 2
max_noice= 1.0
noice_decay = 0.995
training_mode = True
random_seed = 0

In [ ]:
# Initialize Feed-forward DNNs for Actor and Critic models. 
agent = Agent(state_size, action_size, random_seed, max_noice, noice_decay)

#train the agent
def ddpg_train(n_episodes):
    #list containing scores from each episode
    scores = []
    #list containing means over last 100 episodes
    means = []
    #last 100 scores
    scores_window = deque(maxlen = 100)
    # messure time for achieving a mean score over 30
    start_time = time.time()
    for episode in range(n_episodes):
        # messure time for one episode
        start_episode = time.time()
        # Reset the enviroment
        env_info = env.reset(train_mode=training_mode)[brain_name] 
        cur_states = env_info.vector_observations
        score = np.zeros(num_agents)
        # initialize timestep
        timestep = 0
        # reset noise
        agent.reset()
        while True:
            # Choose best action for given network
            actions = agent.act(cur_states, add_noise = True)
            # Action is performed and new state, reward, info are received. 
            env_info = env.step(actions)[brain_name]
            # get next state 
            next_states = env_info.vector_observations
            # see if episode is finished
            dones = env_info.local_done
            # get reward
            rewards = env_info.rewards
            # save experience to replay buffer, perform learning step at defined interval"
            for cur_state, action, reward, next_state, done in zip(cur_states, 
                                                                   actions, 
                                                                   rewards, 
                                                                   next_states, 
                                                                   dones):
                # current state, action, reward, new state are stored in the experience replay"
                agent.step(cur_state, action, reward, next_state, done, timestep)
            # roll over new state
            cur_states = next_states      
            #add reward to score
            score += rewards
            # count timestep
            timestep+=1
            
            if np.any(dones):
                break
        
        # calculate time
        time_episode = time.time() - start_episode
        time_entire = time.time() - start_time
        #save most recent score
        scores_window.append(score)
        scores.append(score)
        mean = np.mean(scores_window)
        means.append(mean)
        print("Episode:" + str(episode) + " Score:" + str(np.mean(score)) + 
              " Mean Score(last 100 episodes):" + str(np.mean(scores_window)) +
              " Duration episode:" + time.strftime('%Mm%Ss', time.gmtime(time_episode)) + 
              " Duration training:" + time.strftime('%Dd%Hh%Mm%Ss', time.gmtime(time_entire)))
        
        # save model weights
        if (episode+1) % 10 == 0 or np.mean(scores_window) >= 30:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_Actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_Critic.pth')
            if np.mean(scores_window) >= 30:
                print("Environment solved in " + str(episode) + " episodes. Mean score over all 20 agents " +
                      str(np.mean(scores_window)) + " for the last 100 episodes")
    
    return scores, means

scores, means = ddpg_train(n_episodes)

#plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores, label = "scores")
plt.plot(np.arange(len(means)), means, label = "means")
plt.axhline(y=30, color='r', linestyle='--', label="target")
plt.ylabel('Score')
plt.xlabel('Episode')
plt.legend()
plt.show()

In [ ]:
env.close()